In [10]:
import pandas as pd
import numpy as np
import re
from sklearn import tree, metrics, decomposition, ensemble, model_selection
from sklearn import naive_bayes, linear_model, svm
from utils import clean

In [5]:
train_df = pd.read_csv('train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_df = pd.read_csv('test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [20]:
df = pd.concat([train_df, test_df],ignore_index=True)
df.tail()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
1304,NaN,NaN,S,8.0500,"Spector, Mr. Woolf",0,1305,3,male,0,NaN,A.5. 3236
1305,39.0,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0,1306,1,female,0,NaN,PC 17758
1306,38.5,NaN,S,7.2500,"Saether, Mr. Simon Sivertsen",0,1307,3,male,0,NaN,SOTON/O.Q. 3101262
1307,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0,1308,3,male,0,NaN,359309
1308,NaN,NaN,C,22.3583,"Peter, Master. Michael J",1,1309,3,male,1,NaN,2668


In [23]:
pattern = '.+, (.+?)\..+'
df.loc[:,'Title'] = df.Name.apply(lambda x: re.match(pattern, x)[1])

In [24]:
df.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr


In [25]:
df.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [ ]:
title_mean_age = {}
for i in df.Title.unique(): title_mean_age[i] = df[df['Title']==i].Age.mean()
for i in df[df.Age.isnull()].index: df.loc[i,'Age'] = title_mean_age[df.loc[i,'Title']]

In [2]:
features_train, labels_train, features_test, answers = clean()

/Users/abhijeetmohanty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [3]:
features_train.head()

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Title
0,0.273456,0.0,0.014151,0.0,1.0,0.0,0.125,0.0
1,0.473882,0.5,0.139136,0.0,0.0,1.0,0.125,0.2
2,0.323563,0.0,0.015469,0.0,1.0,1.0,0.000,0.4
3,0.436302,0.0,0.103644,0.0,0.0,1.0,0.125,0.2
4,0.436302,0.0,0.015713,0.0,1.0,0.0,0.000,0.0


In [ ]:
pca = decomposition.PCA(n_components = 6)
reduced_features = pca.fit_transform(features_train)
reduced_test = pca.transform(features_test)

In [ ]:
#====================== AdaBoost (74.16% accuracy)=============================
tree_clf = tree.DecisionTreeClassifier(min_samples_split = 2, max_depth=4)
abc = ensemble.AdaBoostClassifier(base_estimator = tree_clf, n_estimators=80, learning_rate=0.7)
parameters = {'n_estimators': [90], 'learning_rate': [0.9]}
clf = model_selection.GridSearchCV(abc, param_grid = parameters)
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output,answers['Survived'].values)

In [ ]:
#================== Logistic Regression (78.7% accuracy) =====================
clf = linear_model.LogisticRegression(C=100,solver='newton-cg',multi_class='multinomial')
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output, answers['Survived'].values)

In [ ]:
#=========================== SVM (77.75% accuracy) ============================
params = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100]}
clf = model_selection.GridSearchCV(svm.SVC(), param_grid = params)
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output, answers['Survived'].values)

In [ ]:
#============================== Tree =========================================
params = {"max_depth": np.arange(1, 6), "max_features": np.arange(1, 7),
         "min_samples_leaf": np.arange(1, 6), "criterion": ["gini","entropy"]}
clf = model_selection.GridSearchCV(tree.DecisionTreeClassifier(),param_grid = params)
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output, answers['Survived'].values)

In [ ]:
#======================== RandomForestClassifier =============================
params = {'n_estimators':[100,120], 'max_features':[5,6,7]}
clf = model_selection.GridSearchCV(ensemble.RandomForestClassifier(),param_grid = params)
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output, answers['Survived'].values)
#=============================================================================

In [ ]:
#============================ Naive Bayes ====================================
clf = naive_bayes.GaussianNB()
clf.fit(reduced_features, labels_train)
output = clf.predict(reduced_test)
accuracy_score = metrics.accuracy_score(output, answers['Survived'].values)
#=============================================================================